# <font color='CC3D3D'> 0.5_6 Log_Extract_App_Path
    
- log_data.csv, loan_result.csv, user_spec.csv로부터 개별 유저가 하루 동안 앱에 머물면서 발생시킨 이벤트의 순서를 경로로 정의
- 개별 유저가 방문한 경로 데이터를 집계한 뒤 리스트의 형태로 붙인 데이터
- 대출 신청을 실제로 한 유저의 경로 데이터 <span style="color:blue"> **app_path.csv** </span> 생성
- 대출 신청을 실제로 하지 않은 유저의 경로 데이터 <span style="color:blue"> **.noapp_path.csv** </span> 생성

# Import

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import pickle
from IPython.display import display
import gc

pd.set_option('display.max_columns', None)

# with open(f"{PATH}/빅콘테스트/dtypes.pkl", 'rb') as f:
#     dtypes = pickle.load(f)

In [ ]:
def view_info(df:pd.DataFrame) -> None:
    print(df.shape)
    print()
    df.info()
    print()
    display(df.head())

# Data Load

In [ ]:
loan = pd.read_csv("../Data/2022빅콘테스트_데이터분석리그_데이터분석분야_퓨처스부문_데이터셋_220908/loan_result.csv", engine='c',
                   low_memory=True,
                   parse_dates=['loanapply_insert_time'],
                   dtype={'application_id': 'uint32', 'bank_id': 'uint8', 'product_id': 'uint8'},
                   usecols=['application_id', 'loanapply_insert_time', 'is_applied'])
log = pd.read_csv("../Data/2022빅콘테스트_데이터분석리그_데이터분석분야_퓨처스부문_데이터셋_220908/log_data.csv", engine='c', 
                  dtype={'user_id': 'uint32', 'event': 'object'},
                  usecols=['user_id', 'event', 'timestamp', 'date_cd'], 
                  low_memory=True,
                  parse_dates=['timestamp', 'date_cd'])
user = pd.read_csv("../Data/2022빅콘테스트_데이터분석리그_데이터분석분야_퓨처스부문_데이터셋_220908/user_spec.csv", engine='c',
                   dtype={'application_id': 'uint32', 'user_id': 'uint32'},
                   low_memory=True,
                   parse_dates=['insert_time'],
                   usecols=['application_id', 'user_id', 'insert_time'])

view_info(loan)
print()
view_info(log)
print()
view_info(user)

(13527363, 3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13527363 entries, 0 to 13527362
Data columns (total 3 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   application_id         uint32        
 1   loanapply_insert_time  datetime64[ns]
 2   is_applied             float64       
dtypes: datetime64[ns](1), float64(1), uint32(1)
memory usage: 258.0 MB



,application_id,loanapply_insert_time,is_applied
0,1748340,2022-06-07 13:05:41,NaN
1,1748340,2022-06-07 13:05:41,NaN
2,1748340,2022-06-07 13:05:41,NaN
3,1748340,2022-06-07 13:05:41,NaN
4,1748340,2022-06-07 13:05:41,NaN



(17843993, 4)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17843993 entries, 0 to 17843992
Data columns (total 4 columns):
 #   Column     Dtype         
---  ------     -----         
 0   user_id    uint32        
 1   event      object        
 2   timestamp  datetime64[ns]
 3   date_cd    datetime64[ns]
dtypes: datetime64[ns](2), object(1), uint32(1)
memory usage: 476.5+ MB



,user_id,event,timestamp,date_cd
0,576409,StartLoanApply,2022-03-25 11:12:09,2022-03-25
1,576409,ViewLoanApplyIntro,2022-03-25 11:12:09,2022-03-25
2,72878,EndLoanApply,2022-03-25 11:14:44,2022-03-25
3,645317,OpenApp,2022-03-25 11:15:09,2022-03-25
4,645317,UseLoanManage,2022-03-25 11:15:11,2022-03-25



(1394216, 3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1394216 entries, 0 to 1394215
Data columns (total 3 columns):
 #   Column          Non-Null Count    Dtype         
---  ------          --------------    -----         
 0   application_id  1394216 non-null  uint32        
 1   user_id         1394216 non-null  uint32        
 2   insert_time     1394216 non-null  datetime64[ns]
dtypes: datetime64[ns](1), uint32(2)
memory usage: 21.3 MB



,application_id,user_id,insert_time
0,1249046,118218,2022-06-07 06:28:18
1,954900,553686,2022-06-07 14:29:03
2,137274,59516,2022-06-07 21:40:22
3,1570936,167320,2022-06-07 09:40:27
4,967833,33400,2022-06-07 08:55:07


In [ ]:
loan.isnull().sum()

application_id                 0
loanapply_insert_time          0
is_applied               3257239
dtype: int64

# Data Structure

In [ ]:
app_user = {user['application_id'].iloc[i]: user['user_id'].iloc[i] for i in range(user.shape[0])}
# train = loan[loan['loanapply_insert_time'].dt.month < 6]
# train['user_id'] = train['application_id'].map(app_user)
# train.dropna(subset=['user_id'], inplace=True)
# train['user_id'] = train['user_id'].astype(np.uint32)

# view_info(train)

In [ ]:
train = loan[loan['loanapply_insert_time'].dt.month < 6]
apply_group = train[['application_id', 'is_applied']].groupby(by='application_id').sum()
apply_group.reset_index(drop=False, inplace=True)
apply_group.columns = ['application_id', 'total_applied']

view_info(apply_group)

(728997, 2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 728997 entries, 0 to 728996
Data columns (total 2 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   application_id  728997 non-null  uint64 
 1   total_applied   728997 non-null  float64
dtypes: float64(1), uint64(1)
memory usage: 11.1 MB



,application_id,total_applied
0,1,0.0
1,2,0.0
2,6,0.0
3,14,3.0
4,17,1.0


In [ ]:
app_id = apply_group[apply_group['total_applied'] != 0]['application_id'].unique().astype(np.uint32)
noapp_id = apply_group[apply_group['total_applied'] == 0]['application_id'].unique().astype(np.uint32)

print(f"# of application ids that applied: {app_id.shape[0]}")
print(f"# of application ids that not applied: {noapp_id.shape[0]}")

print(app_id.max())
print(noapp_id.max())

# of application ids that applied: 377669
# of application ids that not applied: 351328
2167826
2167819


In [ ]:
total_loan = pd.merge(left=train, right=apply_group, on='application_id', how='inner')
view_info(total_loan)

(10270124, 4)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10270124 entries, 0 to 10270123
Data columns (total 4 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   application_id         uint32        
 1   loanapply_insert_time  datetime64[ns]
 2   is_applied             float64       
 3   total_applied          float64       
dtypes: datetime64[ns](1), float64(2), uint32(1)
memory usage: 352.6 MB



,application_id,loanapply_insert_time,is_applied,total_applied
0,2157865,2022-05-09 08:44:59,1.0,1.0
1,576643,2022-05-09 10:54:53,0.0,0.0
2,576643,2022-05-09 10:54:53,0.0,0.0
3,2136706,2022-05-09 10:41:06,0.0,3.0
4,2136706,2022-05-09 10:41:07,0.0,3.0


In [ ]:
total_loan['user_id'] = total_loan['application_id'].map(app_user)
total_loan.dropna(subset=['user_id'], inplace=True)
total_loan['user_id'] = total_loan['user_id'].astype(np.uint32)
total_loan['date_cd'] = pd.to_datetime(total_loan['loanapply_insert_time'].dt.date)
view_info(total_loan)

(10270011, 6)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10270011 entries, 0 to 10270123
Data columns (total 6 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   application_id         uint32        
 1   loanapply_insert_time  datetime64[ns]
 2   is_applied             float64       
 3   total_applied          float64       
 4   user_id                uint32        
 5   date_cd                datetime64[ns]
dtypes: datetime64[ns](2), float64(2), uint32(2)
memory usage: 470.1 MB



,application_id,loanapply_insert_time,is_applied,total_applied,user_id,date_cd
0,2157865,2022-05-09 08:44:59,1.0,1.0,346970,2022-05-09
1,576643,2022-05-09 10:54:53,0.0,0.0,545882,2022-05-09
2,576643,2022-05-09 10:54:53,0.0,0.0,545882,2022-05-09
3,2136706,2022-05-09 10:41:06,0.0,3.0,558819,2022-05-09
4,2136706,2022-05-09 10:41:07,0.0,3.0,558819,2022-05-09


In [ ]:
log_train = log[log['date_cd'].dt.month < 6]
view_info(log_train)

(12907328, 4)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12907328 entries, 0 to 17843992
Data columns (total 4 columns):
 #   Column     Dtype         
---  ------     -----         
 0   user_id    uint32        
 1   event      object        
 2   timestamp  datetime64[ns]
 3   date_cd    datetime64[ns]
dtypes: datetime64[ns](2), object(1), uint32(1)
memory usage: 443.1+ MB



,user_id,event,timestamp,date_cd
0,576409,StartLoanApply,2022-03-25 11:12:09,2022-03-25
1,576409,ViewLoanApplyIntro,2022-03-25 11:12:09,2022-03-25
2,72878,EndLoanApply,2022-03-25 11:14:44,2022-03-25
3,645317,OpenApp,2022-03-25 11:15:09,2022-03-25
4,645317,UseLoanManage,2022-03-25 11:15:11,2022-03-25


In [ ]:
app_log_merge = pd.merge(left=log_train, right=total_loan[['user_id', 'application_id', 'date_cd', 'total_applied']], on=['user_id', 'date_cd'], how='inner')
app_log_merge['total_applied'] = app_log_merge['total_applied'].astype(np.uint8)
view_info(app_log_merge)

(105414947, 6)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 105414947 entries, 0 to 105414946
Data columns (total 6 columns):
 #   Column          Dtype         
---  ------          -----         
 0   user_id         uint32        
 1   event           object        
 2   timestamp       datetime64[ns]
 3   date_cd         datetime64[ns]
 4   application_id  uint32        
 5   total_applied   uint8         
dtypes: datetime64[ns](2), object(1), uint32(2), uint8(1)
memory usage: 4.0+ GB



,user_id,event,timestamp,date_cd,application_id,total_applied
0,576409,StartLoanApply,2022-03-25 11:12:09,2022-03-25,87171,0
1,576409,StartLoanApply,2022-03-25 11:12:09,2022-03-25,87171,0
2,576409,StartLoanApply,2022-03-25 11:12:09,2022-03-25,87171,0
3,576409,StartLoanApply,2022-03-25 11:12:09,2022-03-25,87171,0
4,576409,StartLoanApply,2022-03-25 11:12:09,2022-03-25,87171,0


In [ ]:
actual_log = app_log_merge[app_log_merge['application_id'].isin(app_id)]
no_log = app_log_merge[app_log_merge['application_id'].isin(noapp_id)]

In [ ]:
view_info(actual_log)

(59026370, 6)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59026370 entries, 42 to 105414946
Data columns (total 6 columns):
 #   Column          Dtype         
---  ------          -----         
 0   user_id         uint32        
 1   event           object        
 2   timestamp       datetime64[ns]
 3   date_cd         datetime64[ns]
 4   application_id  uint32        
 5   total_applied   uint8         
dtypes: datetime64[ns](2), object(1), uint32(2), uint8(1)
memory usage: 2.3+ GB



,user_id,event,timestamp,date_cd,application_id,total_applied
42,72878,EndLoanApply,2022-03-25 11:14:44,2022-03-25,871260,2
43,72878,EndLoanApply,2022-03-25 11:14:44,2022-03-25,871260,2
44,72878,EndLoanApply,2022-03-25 11:14:44,2022-03-25,871260,2
45,72878,EndLoanApply,2022-03-25 11:14:44,2022-03-25,871260,2
46,72878,EndLoanApply,2022-03-25 11:14:44,2022-03-25,871260,2


In [ ]:
view_info(no_log)

(46388577, 6)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 46388577 entries, 0 to 105414922
Data columns (total 6 columns):
 #   Column          Dtype         
---  ------          -----         
 0   user_id         uint32        
 1   event           object        
 2   timestamp       datetime64[ns]
 3   date_cd         datetime64[ns]
 4   application_id  uint32        
 5   total_applied   uint8         
dtypes: datetime64[ns](2), object(1), uint32(2), uint8(1)
memory usage: 1.8+ GB



,user_id,event,timestamp,date_cd,application_id,total_applied
0,576409,StartLoanApply,2022-03-25 11:12:09,2022-03-25,87171,0
1,576409,StartLoanApply,2022-03-25 11:12:09,2022-03-25,87171,0
2,576409,StartLoanApply,2022-03-25 11:12:09,2022-03-25,87171,0
3,576409,StartLoanApply,2022-03-25 11:12:09,2022-03-25,87171,0
4,576409,StartLoanApply,2022-03-25 11:12:09,2022-03-25,87171,0


In [ ]:
del app_log_merge
gc.collect()

123

In [ ]:
def get_path(df:pd.DataFrame) -> pd.DataFrame:
    df.sort_values(by=['application_id', 'timestamp'], inplace=True)
    event_tuples = dict()
    for appid in tqdm(df['application_id'].unique()):
        subdf = df[df['application_id'] == appid]
        for dt in subdf['date_cd'].unique():
            subsubdf = subdf[subdf['date_cd'] == dt]
            events = tuple(subsubdf['event'])
            if events not in event_tuples:
                event_tuples[events] = 1
            else:
                event_tuples[events] += 1

    result = pd.DataFrame({'path': event_tuples.keys(), 'count': event_tuples.values()})

    return result

## Deployment CSV

In [ ]:
app_path = get_path(actual_log)
app_path.to_csv("../Data/path_applied_user.csv", index=False)

100%|██████████| 354998/354998 [6:08:59<00:00, 16.03it/s]


In [ ]:
noapp_path = get_path(no_log)
noapp_path.to_csv("../Data/path_no_applied_user.csv", index=False)

100%|██████████| 323291/323291 [4:33:21<00:00, 19.71it/s]


In [ ]:
app_path.head(10)

,path,count
0,"(GetCreditInfo, GetCreditInfo, GetCreditInfo, ...",172
1,"(OpenApp, ViewLoanApplyIntro, CompleteIDCertif...",1
2,"(OpenApp, OpenApp, OpenApp, OpenApp, OpenApp, ...",27
3,"(OpenApp, StartLoanApply, ViewLoanApplyIntro, ...",2526
4,"(OpenApp, OpenApp, OpenApp, OpenApp, OpenApp, ...",4
5,"(OpenApp, StartLoanApply, ViewLoanApplyIntro, ...",65
6,"(OpenApp, OpenApp, OpenApp, OpenApp, OpenApp, ...",1
7,"(Login, Login, Login, Login, Login, GetCreditI...",4
8,"(OpenApp, OpenApp, Login, ViewLoanApplyIntro, ...",2
9,"(OpenApp, OpenApp, OpenApp, OpenApp, OpenApp, ...",9


In [ ]:
app_path.sort_values(by='count', ascending=False, inplace=True)
app_path.reset_index(drop=True, inplace=True)
for i in range(20):
    path, count = app_path.head(20).iloc[i]
    print(f"{path}\t{count}")

('OpenApp', 'Login', 'StartLoanApply', 'ViewLoanApplyIntro', 'CompleteIDCertification', 'EndLoanApply')	6565
('OpenApp', 'OpenApp', 'Login', 'Login', 'StartLoanApply', 'StartLoanApply', 'ViewLoanApplyIntro', 'ViewLoanApplyIntro', 'CompleteIDCertification', 'CompleteIDCertification', 'EndLoanApply', 'EndLoanApply')	4169
('OpenApp', 'OpenApp', 'OpenApp', 'Login', 'Login', 'Login', 'StartLoanApply', 'StartLoanApply', 'StartLoanApply', 'ViewLoanApplyIntro', 'ViewLoanApplyIntro', 'ViewLoanApplyIntro', 'CompleteIDCertification', 'CompleteIDCertification', 'CompleteIDCertification', 'EndLoanApply', 'EndLoanApply', 'EndLoanApply')	2676
('OpenApp', 'StartLoanApply', 'ViewLoanApplyIntro', 'CompleteIDCertification', 'EndLoanApply')	2526
('OpenApp', 'OpenApp', 'OpenApp', 'OpenApp', 'Login', 'Login', 'Login', 'Login', 'StartLoanApply', 'StartLoanApply', 'StartLoanApply', 'StartLoanApply', 'ViewLoanApplyIntro', 'ViewLoanApplyIntro', 'ViewLoanApplyIntro', 'ViewLoanApplyIntro', 'CompleteIDCertificatio

In [ ]:
noapp_path.head(10)

,path,count
0,"(OpenApp, OpenApp, OpenApp, OpenApp, OpenApp, ...",20
1,"(OpenApp, Login, StartLoanApply, ViewLoanApply...",105
2,"(OpenApp, OpenApp, OpenApp, OpenApp, OpenApp, ...",2
3,"(OpenApp, OpenApp, OpenApp, OpenApp, OpenApp, ...",2
4,"(OpenApp, OpenApp, OpenApp, OpenApp, OpenApp, ...",318
5,"(OpenApp, OpenApp, OpenApp, OpenApp, OpenApp, ...",13
6,"(OpenApp, OpenApp, OpenApp, OpenApp, OpenApp, ...",1
7,"(OpenApp, Login, StartLoanApply, ViewLoanApply...",1
8,"(GetCreditInfo, GetCreditInfo, GetCreditInfo, ...",157
9,"(OpenApp, OpenApp, OpenApp, OpenApp, Login, Lo...",2465


In [ ]:
noapp_path.sort_values(by='count', ascending=False, inplace=True)
noapp_path.reset_index(drop=True, inplace=True)
for i in range(20):
    path, count = noapp_path.head(20).iloc[i]
    print(f"{path}\t{count}")

('OpenApp', 'Login', 'StartLoanApply', 'ViewLoanApplyIntro', 'CompleteIDCertification', 'EndLoanApply')	8376
('OpenApp', 'OpenApp', 'Login', 'Login', 'StartLoanApply', 'StartLoanApply', 'ViewLoanApplyIntro', 'ViewLoanApplyIntro', 'CompleteIDCertification', 'CompleteIDCertification', 'EndLoanApply', 'EndLoanApply')	4879
('OpenApp', 'OpenApp', 'OpenApp', 'Login', 'Login', 'Login', 'StartLoanApply', 'StartLoanApply', 'StartLoanApply', 'ViewLoanApplyIntro', 'ViewLoanApplyIntro', 'ViewLoanApplyIntro', 'CompleteIDCertification', 'CompleteIDCertification', 'CompleteIDCertification', 'EndLoanApply', 'EndLoanApply', 'EndLoanApply')	3319
('OpenApp', 'StartLoanApply', 'ViewLoanApplyIntro', 'CompleteIDCertification', 'EndLoanApply')	3031
('OpenApp', 'OpenApp', 'OpenApp', 'OpenApp', 'Login', 'Login', 'Login', 'Login', 'StartLoanApply', 'StartLoanApply', 'StartLoanApply', 'StartLoanApply', 'ViewLoanApplyIntro', 'ViewLoanApplyIntro', 'ViewLoanApplyIntro', 'ViewLoanApplyIntro', 'CompleteIDCertificatio